In [37]:
ls

2016ElectionDataByState.csv
Cleaning Data - US President & Vice President.ipynb
General2016.csv
README.md
__pycache__/
config.py
proposal.txt
replies.py


In [38]:
import pandas as pd
import tweepy
import time
import json
from replies import *
from pprint import pprint
from config import consumer_key, consumer_secret, access_token, access_token_secret

In [39]:
# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [40]:
#import csv
file = "2016ElectionDataByState.csv"
df = pd.read_csv(file)
del df['Unnamed: 3']
df


,State,Clinton,Trump
0,Alabama,"729,547","1,318,255"
1,Alaska,"116,454","163,387"
2,Arizona,"1,161,167","1,252,401"
3,Arkansas,"380,494","684,872"
4,California,"8,753,792","4,483,814"
5,Colorado,"1,338,870","1,202,484"
6,Connecticut,"897,572","673,215"
7,Delaware,"235,603","185,127"
8,D. C.,"282,830","12,723"
9,Florida,"4,504,975","4,617,886"


In [41]:
# #show US Pres & Vice Pres only
# clean_df = df.loc[(df["Race"] == 'US President & Vice President')]
# clean_df.head()

In [42]:
# df = clean_df.groupby(["CounterType","Precinct","Party"])["SumOfCount"].sum().reset_index()  
# df.head()

In [43]:
# df.loc[(df["Party"].isin(['RPN','DPN']))].reset_index(drop=True)  

In [44]:
# Target Term
target_user = "@realDonaldTrump"

In [46]:
# Get all tweets from target user
tweet_list = []
public_tweets = api.user_timeline(target_user, page=1)
original_tweet = 0
# Loop through all tweets
for i, tweet in enumerate(public_tweets):
    tweet_list.append(tweet)
    replies_list = []
    orginal_tweet = tweet.get('id')

    replies = api.search(q='to:@realDonaldTrump',since_id=tweet.get('id'),count=100,geocode='41.8781,-87.6298,2000mi')
    while len(replies.get('statuses')) > 0 and replies.get('statuses')[0].get('id') > original_tweet:
        for reply in replies["statuses"]:
#             if reply.get('coordinates') != None:
            replies_list.append(reply) 
        replies = api.search(q='to:@realDonaldTrump',since_id=reply.get('id'), count=100,geocode='41.8781,-87.6298,2000mi')
        original_tweet = reply.get('id')  
        if (len(replies_list) > 4000):
            break
    break

    

In [47]:
len(tweet_list), len(replies_list)

(1, 4027)

In [14]:
pprint(tweet_list[0].get('id')), pprint(tweet_list[0].get('created_at')), pprint(tweet_list[0].get('text'))

1004478326336040960
'Wed Jun 06 21:41:09 +0000 2018'
('Thank you to everyone at @FEMA HQ for today’s briefing on preparations for '
 'the upcoming hurricane season. Disaster… https://t.co/SszyuwMdWp')


(None, None, None)

In [53]:
# for reply in replies_list:
#     pprint(reply.get('id'))
#     pprint(reply.get('created_at'))
#     pprint(reply.get('text'))
#     pprint(reply.get('coordinates'))
#     break
    
replies_df = pd.DataFrame(replies_list)



In [97]:
replies_list[0].get('user').get('location')
replies_list[0]

{'contributors': None,
 'coordinates': None,
 'created_at': 'Sat Jun 09 15:59:43 +0000 2018',
 'entities': {'hashtags': [],
  'symbols': [],
  'urls': [{'display_url': 'twitter.com/IkeMagnifico/s…',
    'expanded_url': 'https://twitter.com/IkeMagnifico/status/1005471397601251329',
    'indices': [17, 40],
    'url': 'https://t.co/6kUUAgpZ1Q'}],
  'user_mentions': [{'id': 25073877,
    'id_str': '25073877',
    'indices': [0, 16],
    'name': 'Donald J. Trump',
    'screen_name': 'realDonaldTrump'}]},
 'favorite_count': 0,
 'favorited': False,
 'geo': None,
 'id': 1005479565391691776,
 'id_str': '1005479565391691776',
 'in_reply_to_screen_name': 'realDonaldTrump',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': 25073877,
 'in_reply_to_user_id_str': '25073877',
 'is_quote_status': True,
 'lang': 'und',
 'metadata': {'iso_language_code': 'und', 'result_type': 'recent'},
 'place': None,
 'possibly_sensitive': False,
 'quoted_status': {'contributo

In [113]:
flat_replies = []
for reply in replies_list:
    if len(reply.get('user').get('location')) > 0:
        reply_dict = {'id': reply.get('id'),
                    'text' : reply.get('text'),
                    'location': reply.get('user').get('location')}
        flat_replies.append(reply_dict)

In [114]:
replies_df = pd.DataFrame(flat_replies)
replies_df

,id,location,text
0,1005479565391691776,"New Jersey, USA",@realDonaldTrump https://t.co/6kUUAgpZ1Q
1,1005479550099382272,"Glen Head, NY",@realDonaldTrump Go get ‘em
2,1005479525331931136,Österreich-Südostasien,RT @therealamydavis: @realDonaldTrump Yeah.......
3,1005479487247667200,"Kokomo, Indiana",@realDonaldTrump https://t.co/MvEkJsM25E
4,1005479468197208065,"Los Angeles, CA",@realDonaldTrump Who are you to say that it is...
5,1005479428493860867,"Pennsylvania, USA",@realDonaldTrump Just thought I would let you ...
6,1005479422378426368,United States,@realDonaldTrump is an ignorant fool! https://...
7,1005479378254524416,United States,@realDonaldTrump Helps Putin https://t.co/f011...
8,1005479358914613250,United States,@realDonaldTrump #FairTrade #MorePeaceLessWar ...
9,1005479350941020161,Arizona/Mexico Border,"RT @walthers87: @realDonaldTrump It's like, no..."


In [110]:
replies_df.head()

,id,location,text
0,1005479565391691776,"New Jersey, USA",@realDonaldTrump https://t.co/6kUUAgpZ1Q
1,1005479550099382272,"Glen Head, NY",@realDonaldTrump Go get ‘em
2,1005479525331931136,Österreich-Südostasien,RT @therealamydavis: @realDonaldTrump Yeah.......
3,1005479487872688129,,"RT @USAloveGOD: @realDonaldTrump Yes, may our ..."
4,1005479487247667200,"Kokomo, Indiana",@realDonaldTrump https://t.co/MvEkJsM25E
